---
# Apresentação
---
## Resumo
> Suponha uma empresa de turismo em uma grande cidade e que há um turista que deseja conhecer essa cidade. Entretanto, ele elencou apenas 1 ponto que deseja visitar. A proposta do projeto é desenvolver um algoritmo capaz de gerar uma rota a partir da sede da empresa e que passe por uma quantidade mínima de pontos turísticos, de modo que o ponto final seja o ponto escolhido pelo cliente. Assim, ele poderá conhecer mais lugares da cidade, além daquele previamente escolhido.

## Objetivo
> O presente arquivo visa utilizar a API da Plataforma Google Maps para extrair os dados dos pontos turísticos, os quais serão utilizados para o desenvolvimento do projeto descrito.

## Notas
> Este Código foi desenvolvido em Python no Google Colaboratory e seu funcionamento está a ele relacionado.

> Acesse a [documentação completa](https://developers.google.com/maps/documentation/places/web-service?hl=pt-br) da API Places.

> **Para um melhor acompanhamento**, [visualize através do Colab](https://colab.research.google.com/drive/1kC_7_iHB3_C8XIOqyBHwcpdKS0YokC5v?usp=sharing) utilizando os índices.

---
# Bibliotecas, funções e constantes
---

In [4]:
# @title Importando as bibliotecas necessárias

# API
import requests

# Matemática
import math
import numpy as np
import pandas as pd

# Manipulação
import json
from datetime import datetime
from binascii import Error
from time import sleep
from pprint import pprint

In [5]:
# @title Definindo a chave da API

API_KEY = '#'

In [6]:
# @title Função para obter a geocodificação de um ponto

def obtem_geocodificacao(endereco: str) -> tuple[float, float]:
  """
    Obtem a geocodificação (Latitude e Longitude) de um determinado endereço do
    Google maps, caso não seja possível, um Erro é lançado.
  """
  base_url = 'https://maps.googleapis.com/maps/api/geocode/json'

  params = {
      'key': API_KEY,
      'address': endereco,
      'language': 'pt-BR'
  }
  response = requests.post(base_url, params=params)
  resultado = response.json()

  if resultado['status'] == "OK":
    location = resultado['results'][0]['geometry']['location']
    lat, lng = float(location['lat']), float(location['lng'])
    return (lat, lng)

  else:
    raise Error(f'Erro na api do google maps: {resultado["error_message"]}')

In [7]:
# @title Função para obter a distância euclidiana entre dois pontos


def distancia_euclidiana_em_km(lat1: float, lon1: float , lat2: float , lon2: float ) -> float:
  """
    Calcula a distância euclidiana (aproximada em quilômetros) entre dois pontos geográficos, dados suas latitudes e longitudes
    Material de referência:
    https://cref.if.ufrgs.br/?contact-pergunta=calculo-aproximado-de-distancias-com-base-em-coordenadas-de-latitude-e-longitude
  """
  lat_km_per_degree = 111.1
  lon_km_per_degree = lat_km_per_degree * math.cos(math.radians((lat1 + lat2) / 2))
  delta_lat = (lat2 - lat1) * lat_km_per_degree
  delta_lon = (lon2 - lon1) * lon_km_per_degree
  distancia_euclidiana = math.sqrt(delta_lat**2 + delta_lon**2)
  return distancia_euclidiana

In [8]:
# @title Função para extrair pontos turisticos dentro de um determinado raio

def extrai_pontos_turisticos(lat: float, lng: float, raio: float, quantidade: int = 40) -> pd.DataFrame:
  """
    Extrai determinada quantidade de pontos turisticos dentro de um raio a partir de um ponto (latitude, longitude).

    Caso a quantidade não seja especificada, 40 pontos serão buscados por padrão.

    Caso haja algum erro com a API, um Erro é lançado.
  """
  base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

  params = {
      'location': f"{lat},{lng}",
      'radius': raio,
      'key': API_KEY,
      'language': 'pt-BR',
      'type': 'tourist_attraction'
  }

  pontos_turisticos = []

  while True:
    response = requests.post(base_url, params=params)
    resultados = response.json()

    if resultados['status'] != 'OK':
      raise Error(f'Erro na api do google maps: {resultados["error_message"]}')

    pontos_turisticos.extend(resultados['results'])

    if (len(pontos_turisticos) >= quantidade):
      break

    # Se há um token de próxima página na resposta, significa que há mais resultados disponíveis para busca
    if 'next_page_token' in resultados:
      next_token = resultados['next_page_token']
      params['pagetoken'] = next_token
      sleep(2)
    else:
      break

  return pontos_turisticos


In [9]:
# @title Função para obter a distância estimada entre todos os pontos - h(n)

def obtem_distancia_estimada_entre_todos_os_pontos(todos_os_pontos: pd.DataFrame) -> pd.DataFrame:
  """
    Obtem a distancia estimada de cada ponto para cada ponto turistico.

    Esta função utiliza a função de distância euclidiana aproximada para Km.

    Caso haja algum erro com a API, um Erro é lançado.
  """
  matriz_distancia_euclidiana_km = pd.DataFrame(index=df['Place ID'], columns=df['Place ID'])

  # Iterando sobre cada linha do dataFrame (cada ponto turístico)
  for i, linha_i in df.iterrows():
    for j, linha_j in df.iterrows():
      # Se os pontos turísticos forem diferentes, a distância entre eles é calculada e inserida na matriz
      if i != j:
          dist = distancia_euclidiana_em_km(linha_i['Latitude'], linha_i['Longitude'], linha_j['Latitude'], linha_j['Longitude'])
          matriz_distancia_euclidiana_km.at[linha_i['Place ID'], linha_j['Place ID']] = dist
      # Se os pontos forem os mesmos, a distância é definida como 0
      else:
          matriz_distancia_euclidiana_km.at[linha_i['Place ID'], linha_j['Place ID']] = 0

  return matriz_distancia_euclidiana_km

In [10]:
# @title Função para obter a distância real e tempo real entre todos os pontos - g(n)

def obtem_distancia_e_tempo_real_entre_todos_os_pontos(todos_os_pontos: pd.DataFrame, meio_de_transporte: str) -> tuple[pd.DataFrame, pd.DataFrame]:
  """
    Obtem a distancia e o tempo real gasto de cada ponto para cada ponto turistico.

    Essa função utilizada técnica de lotes respeitando o máximo de pontos permitidos por requisição da API (25 pontos).

    Caso haja algum erro com a API, um Erro é lançado.
  """
  base_url = "https://maps.googleapis.com/maps/api/distancematrix/json"

  # Criando uma nova coluna no dataFrame para facilitar o envio para a API
  todos_os_pontos['coord'] = todos_os_pontos['Latitude'].astype(str) + ',' + todos_os_pontos['Longitude'].astype(str)

  params = {
    'origins': None,
    'destinations': None,
    'mode': meio_de_transporte,
    'key': API_KEY,
    'language': 'pt-BR'
  }

  quantidade_total_de_pontos = len(todos_os_pontos)
  maximo_por_request = 25

  # Inicializando um dataFrame para armazenar a matriz de distância e tempo, com
  # os IDs dos pontos turísticos como índices e colunas
  matriz_distancia_por_carro = pd.DataFrame(index=todos_os_pontos['Place ID'], columns=todos_os_pontos['Place ID'])
  matriz_tempo_por_carro = pd.DataFrame(index=todos_os_pontos['Place ID'], columns=todos_os_pontos['Place ID'])

  for i, linha_i in todos_os_pontos.iterrows():
    # Agrupando os destinos em lotes, de acordo com o número máximo permitido por solicitação
    lotes_de_destinos = [
        todos_os_pontos['coord'].iloc[i:i + maximo_por_request].tolist()
        for i in range(0, quantidade_total_de_pontos, maximo_por_request)
    ]
    lotes_de_place_id = [
        todos_os_pontos['Place ID'].iloc[i:i + maximo_por_request].tolist()
        for i in range(0, quantidade_total_de_pontos, maximo_por_request)
    ]

    params['origins'] = linha_i['coord']
    for lote, lote_place_id in zip(lotes_de_destinos, lotes_de_place_id):
      params['destinations']  = '|'.join(lote) # Formatando destinos para a API

      response = requests.get(base_url, params=params)
      resultado = response.json()

      if resultado['status'] != 'OK':
        raise Error(f'Erro na api do google maps: {resultado["error_message"]}; Erro na linha {i} do lote {lote}')

      origem = resultado['origin_addresses'][0]
      for destino, resposta_api, place_id in zip(resultado['destination_addresses'], resultado['rows'][0]['elements'], lote_place_id):
          distancia = tempo = 0
          # Se a origem é igual ao destino, a distancia e tempo de viagem são 0
          # Senão, obtem a distancia e tempo, convertendo para quilômetros e horas, respectivamente
          if origem == destino:
              distancia = tempo = 0
          else:
              distancia_em_metros = resposta_api['distance']['value']
              tempo_em_segundos = resposta_api['duration']['value']

              distancia_em_km = float(distancia_em_metros) / 1000.0
              tempo_em_horas = float(tempo_em_segundos) / 3600.0

              distancia = distancia_em_km
              tempo = tempo_em_horas

          matriz_distancia_por_carro.at[linha_i['Place ID'], place_id] = distancia
          matriz_tempo_por_carro.at[linha_i['Place ID'], place_id] = tempo

  return matriz_distancia_por_carro, matriz_tempo_por_carro


---
# Extração de dados
---

In [11]:
# @title Definindo as Constantes

# Endereço da sede da empresa de turismo (ponto inicial)
SEDE_LOCALIZACAO = 'R. Barão de São Francisco, 236 - Vila Isabel, Rio de Janeiro - RJ'

# Latitude e Longitude da sede da empresa
LAT, LNG = obtem_geocodificacao(SEDE_LOCALIZACAO)

# Raio de distância da sede onde serão buscados pontos turísticos (em metros)
RAIO = 10000

QUANTIDADE_MAXIMA_DE_PONTOS = 40

# Cidade onde serão buscados os pontos turísticos
CIDADE = "rio"

# Modo de transporte utilizado para a rota
MODO_DE_TRANSPORTE = 'driving'

In [12]:
# @title Obtendo lista de pontos turísticos próximos à empresa
pontos_turisticos = extrai_pontos_turisticos(LAT, LNG, RAIO, QUANTIDADE_MAXIMA_DE_PONTOS)

In [13]:
# @title Exportando os dados obtidos
# Será gerado um DataFrame e um arquivo .csv (Excel)

# Preparando os dados para o DataFrame
pontos_turisticos_filtrado = [{
  'Name': ponto['name'],
  'Latitude': ponto['geometry']['location']['lat'],
  'Longitude': ponto['geometry']['location']['lng'],
  'Place ID': ponto['place_id'],
  'Rating': ponto['rating'],
  'User Ratings Total': ponto['user_ratings_total'],
  'Address': ponto['vicinity'],
  'Types': ', '.join(ponto['types'])
  } for ponto in pontos_turisticos if ponto['business_status'] == 'OPERATIONAL']

# Criando o DataFrame
df_pontos = pd.DataFrame(pontos_turisticos_filtrado)

# Incluindo ponto inicial
df_sede = pd.DataFrame([{
    'Name': 'Sede da empresa (Ponto inicial)',
    'Latitude': LAT,
    'Longitude': LNG,
    'Place ID': "ID_GERADO",
    'Rating': None,
    'User Ratings Total': None,
    'Address': SEDE_LOCALIZACAO,
    'Types': None
}])

df = pd.concat([df_sede, df_pontos], ignore_index=True)

# Salvando em um arquivo Excel
df.to_excel(f'pontos_turisticos-{CIDADE}.xlsx', index=False)

In [14]:
display(df.head())

,Name,Latitude,Longitude,Place ID,Rating,User Ratings Total,Address,Types
0,Sede da empresa (Ponto inicial),-22.919327,-43.250186,ID_GERADO,None,None,"R. Barão de São Francisco, 236 - Vila Isabel, ...",None
1,Praça da República - Campo de Santana,-22.906800,-43.188542,ChIJ-SApQ2l_mQARnALEEfr3Yy8,4.1,12094,"Centro, Rio de Janeiro","park, tourist_attraction, point_of_interest, e..."
2,Cristo Redentor,-22.951916,-43.210487,ChIJP6FKmNV_mQAR3gKVAdeEyZ0,4.8,118621,"Parque Nacional da Tijuca - Alto da Boa Vista,...","tourist_attraction, point_of_interest, establi..."
3,Theatro Municipal do Rio de Janeiro,-22.909048,-43.176558,ChIJUSQw7WF_mQARQKXFPmK9_6A,4.8,19776,"Praça Floriano, S/N - Centro, Rio de Janeiro","tourist_attraction, point_of_interest, establi..."
4,Maracanã,-22.912161,-43.230171,ChIJDYuIul1-mQAR5XRi7ogx8_U,4.7,103327,"Rua Professor Eurico Rabelo - Maracanã, Rio de...","stadium, tourist_attraction, point_of_interest..."


---
# Cálculo de distâncias e tempo de viagem entre os pontos turísticos
---

In [15]:
# @title Mapeando a distância estimada entre todos os pontos

matriz_distancia_euclidiana = obtem_distancia_estimada_entre_todos_os_pontos(todos_os_pontos=df)

display(matriz_distancia_euclidiana.head())

Place ID,ID_GERADO,ChIJ-SApQ2l_mQARnALEEfr3Yy8,ChIJP6FKmNV_mQAR3gKVAdeEyZ0,ChIJUSQw7WF_mQARQKXFPmK9_6A,ChIJDYuIul1-mQAR5XRi7ogx8_U,ChIJf1woiGR_mQARyXRjaTaeIBc,ChIJ0QTJyWF_mQAR4t3oNko2kQo,ChIJA0tgM6PVmwARYbqb7Eo9VO4,ChIJaXW-F2R_mQARjtqcwUcZSfM,ChIJXbKphbbVmwARkyFh0LoSTOY,...,ChIJGx7BXoh-mQARK_B4SbrTjTI,ChIJ0d1kUQp-mQARZ17Wa3A9NKc,ChIJfVzFqCPVmwARUBFlJIm6e8k,ChIJ6ZhYkbd9mQARJJ0N13gYnw4,ChIJabwOd29_mQARKGuNjNLqwuM,ChIJIVr_4xB-mQARA3_foQEaiIA,ChIJiS4yHPp-mQARAgzgz4kGrs8,ChIJ3Vt-cOKBmQARV_PYv8SfH2A,ChIJcSsrEEd8mQARzsVXC8nv8uk,ChIJ4zqbyNV_mQAR_BSfApu9sEY
Place ID,,,,,,,,,,,,,,,,,,,,,
ID_GERADO,0,6.460012,5.441316,7.620683,2.197447,7.158725,7.715566,6.312418,7.167557,6.822011,...,2.183044,0.312916,9.949746,2.985584,6.729965,1.738835,3.245864,7.980361,4.908613,5.094413
ChIJ-SApQ2l_mQARnALEEfr3Yy8,6.460012,0,5.492379,1.251597,4.301661,1.021903,1.371128,7.927838,0.903684,8.973396,...,5.381863,6.569937,9.28942,8.528196,0.555099,5.849223,3.454449,1.559693,9.163326,10.836962
ChIJP6FKmNV_mQAR3gKVAdeEyZ0,5.441316,5.492379,0,5.893671,4.854315,5.346846,5.899522,2.454049,5.498708,3.518692,...,6.388135,5.74192,4.716227,8.42665,5.268623,3.726442,5.299569,6.775119,10.204937,7.119472
ChIJUSQw7WF_mQARQKXFPmK9_6A,7.620683,1.251597,5.893671,0,5.497484,0.560382,0.1285,8.344533,0.468073,9.411867,...,6.628885,7.748787,9.156093,9.770977,0.892054,6.865043,4.694575,0.967599,10.406903,11.798324
ChIJDYuIul1-mQAR5XRi7ogx8_U,2.197447,4.301661,4.854315,5.497484,0,5.069491,5.599689,6.520325,5.057458,7.303203,...,1.596858,2.275184,9.570452,4.418021,4.615687,2.311281,1.099097,5.799669,5.644091,7.053464


In [16]:
# @title Mapeando a distância e tempo real gasto entre os pontos

matriz_distancia_por_carro, matriz_tempo_por_carro = obtem_distancia_e_tempo_real_entre_todos_os_pontos(
    todos_os_pontos=df,
    meio_de_transporte=MODO_DE_TRANSPORTE
)

matriz_distancia_por_carro.to_excel(f'matriz_distancia_carro_km-{CIDADE}.xlsx')
matriz_tempo_por_carro.to_excel(f'matriz_tempo_carro_horas-{CIDADE}.xlsx')

In [17]:
matriz_distancia_por_carro.head()

Place ID,ID_GERADO,ChIJ-SApQ2l_mQARnALEEfr3Yy8,ChIJP6FKmNV_mQAR3gKVAdeEyZ0,ChIJUSQw7WF_mQARQKXFPmK9_6A,ChIJDYuIul1-mQAR5XRi7ogx8_U,ChIJf1woiGR_mQARyXRjaTaeIBc,ChIJ0QTJyWF_mQAR4t3oNko2kQo,ChIJA0tgM6PVmwARYbqb7Eo9VO4,ChIJaXW-F2R_mQARjtqcwUcZSfM,ChIJXbKphbbVmwARkyFh0LoSTOY,...,ChIJGx7BXoh-mQARK_B4SbrTjTI,ChIJ0d1kUQp-mQARZ17Wa3A9NKc,ChIJfVzFqCPVmwARUBFlJIm6e8k,ChIJ6ZhYkbd9mQARJJ0N13gYnw4,ChIJabwOd29_mQARKGuNjNLqwuM,ChIJIVr_4xB-mQARA3_foQEaiIA,ChIJiS4yHPp-mQARAgzgz4kGrs8,ChIJ3Vt-cOKBmQARV_PYv8SfH2A,ChIJcSsrEEd8mQARzsVXC8nv8uk,ChIJ4zqbyNV_mQAR_BSfApu9sEY
Place ID,,,,,,,,,,,,,,,,,,,,,
ID_GERADO,0,7.393,21.523,9.569,2.682,8.984,9.359,13.151,8.977,14.245,...,4.803,0.58,16.768,3.949,8.176,3.46,4.897,14.049,6.751,10.976
ChIJ-SApQ2l_mQARnALEEfr3Yy8,8.291,0,13.848,3.684,6.767,3.099,3.474,11.703,3.092,12.797,...,7.698,7.735,15.321,10.161,2.38,7.724,4.917,6.89,12.652,15.481
ChIJP6FKmNV_mQAR3gKVAdeEyZ0,16.23,12.627,0,12.783,14.706,12.575,12.339,11.958,12.339,13.052,...,15.637,15.673,15.576,18.1,11.725,14.095,13.588,15.711,29.567,12.555
ChIJUSQw7WF_mQARQKXFPmK9_6A,9.611,2.828,12.581,0,8.087,0.879,0.802,13.023,1.15,14.117,...,9.018,9.055,12.201,11.481,1.416,9.044,6.237,4.345,13.972,16.801
ChIJDYuIul1-mQAR5XRi7ogx8_U,4.264,4.896,19.026,7.072,0,6.487,6.862,10.654,6.48,11.748,...,4.43,3.707,14.271,6.893,5.679,3.588,2.4,11.553,9.384,11.344


In [18]:
matriz_tempo_por_carro.head()

Place ID,ID_GERADO,ChIJ-SApQ2l_mQARnALEEfr3Yy8,ChIJP6FKmNV_mQAR3gKVAdeEyZ0,ChIJUSQw7WF_mQARQKXFPmK9_6A,ChIJDYuIul1-mQAR5XRi7ogx8_U,ChIJf1woiGR_mQARyXRjaTaeIBc,ChIJ0QTJyWF_mQAR4t3oNko2kQo,ChIJA0tgM6PVmwARYbqb7Eo9VO4,ChIJaXW-F2R_mQARjtqcwUcZSfM,ChIJXbKphbbVmwARkyFh0LoSTOY,...,ChIJGx7BXoh-mQARK_B4SbrTjTI,ChIJ0d1kUQp-mQARZ17Wa3A9NKc,ChIJfVzFqCPVmwARUBFlJIm6e8k,ChIJ6ZhYkbd9mQARJJ0N13gYnw4,ChIJabwOd29_mQARKGuNjNLqwuM,ChIJIVr_4xB-mQARA3_foQEaiIA,ChIJiS4yHPp-mQARAgzgz4kGrs8,ChIJ3Vt-cOKBmQARV_PYv8SfH2A,ChIJcSsrEEd8mQARzsVXC8nv8uk,ChIJ4zqbyNV_mQAR_BSfApu9sEY
Place ID,,,,,,,,,,,,,,,,,,,,,
ID_GERADO,0,0.250278,0.620833,0.367778,0.123333,0.3225,0.374167,0.356667,0.323333,0.408333,...,0.156389,0.041389,0.481667,0.162222,0.334722,0.187778,0.211389,0.382778,0.211667,0.480833
ChIJ-SApQ2l_mQARnALEEfr3Yy8,0.296667,0,0.515,0.195556,0.195,0.15,0.201667,0.3,0.150833,0.351667,...,0.215278,0.266944,0.425,0.299722,0.1575,0.299444,0.187222,0.236389,0.381944,0.61
ChIJP6FKmNV_mQAR3gKVAdeEyZ0,0.673056,0.519722,0,0.583611,0.571389,0.555278,0.571944,0.425833,0.564167,0.477778,...,0.591667,0.643333,0.550833,0.676111,0.535278,0.649722,0.5725,0.571389,0.701111,0.666944
ChIJUSQw7WF_mQARQKXFPmK9_6A,0.365,0.146389,0.553333,0,0.263611,0.060833,0.066111,0.368611,0.081111,0.420278,...,0.283889,0.335556,0.413611,0.368333,0.103889,0.368056,0.255833,0.130278,0.450556,0.678611
ChIJDYuIul1-mQAR5XRi7ogx8_U,0.198611,0.146944,0.517778,0.264722,0,0.219167,0.270833,0.253333,0.220278,0.305278,...,0.125556,0.168889,0.378611,0.209722,0.231667,0.181667,0.108333,0.279722,0.291944,0.492222


---
# Representação do Grafo e Hiperconexão
---

In [19]:
import folium
import pandas as pd

In [20]:
# @title Lendo Dataset
df_pontos = pd.read_excel('./pontos_turisticos-rio.xlsx')

df_distancias = pd.read_excel('./matriz_distancia_carro_km-rio.xlsx', index_col='Place ID')

In [21]:
# @title Função para criar um mapa com marcadores para todos os pontos turístico

def cria_mapa_com_pontos(todos_os_pontos: pd.DataFrame, com_raio=True):
  mapa = folium.Map(location=[df_pontos['Latitude'].mean(), df_pontos['Longitude'].mean()], zoom_start=12)

  for index, row in df_pontos.iterrows():
    if row['Name'] == 'Sede da empresa (Ponto inicial)':
      folium.Marker(
          [row['Latitude'], row['Longitude']],
          popup=row['Name'],
          icon=folium.Icon(color='green',icon_color='#F7F7F7', prefix='fa', icon='building')
      ).add_to(mapa)

      if com_raio:
        # Adiciona um círculo de 10 km de raio ao redor da sede
        folium.Circle(
            radius=10000,  # Raio em metros
            location=[row['Latitude'], row['Longitude']],
            popup='Área de 10 km',
            color='blue',
            fill=True,
            fill_color='blue',
            fill_opacity=0.1
        ).add_to(mapa)
    else:
      folium.Marker(
          [row['Latitude'], row['Longitude']],
          popup=row['Name'],
          icon=folium.Icon(color='red', prefix='fa', icon='camera')
      ).add_to(mapa)

  return mapa

mapa = cria_mapa_com_pontos(df_pontos)
mapa.save("representacao_problema_de_busca.html")
mapa

In [22]:
# @title Função para criar conexoes em um mapa com marcadores

def adiciona_conexoes_em_um_mapa_com_pontos(mapa, grafo, densidade_linha=1):
  for place_id, grafo_ids in grafo.items():
      ponto_origem = df_pontos.loc[df_pontos['Place ID'] == place_id, ['Latitude', 'Longitude']].values[0]
      for conexao_id in grafo_ids:
          ponto_destino = df_pontos.loc[df_pontos['Place ID'] == conexao_id, ['Latitude', 'Longitude']].values[0]
          folium.PolyLine(
              locations=[ponto_origem, ponto_destino],
              color="blue",
              weight=densidade_linha
          ).add_to(mapa)

In [23]:
# @title Representando Hiperconectividade

mapa = cria_mapa_com_pontos(todos_os_pontos=df_pontos)

# Conectando cada ponto com todos os outros pontos
coords = df_pontos[['Latitude', 'Longitude']].values
for i in range(len(coords)):
  for j in range(len(coords)):
    if i != j:
      folium.PolyLine(
          locations=[coords[i], coords[j]],
          color="blue",
          weight=1,
          opacity=0.25
      ).add_to(mapa)

mapa.save("grafo_com_hiper-conectividade.html")
mapa

In [24]:
# @title Constantes e Funções preliminares

NUMERO_MAXIMO_DE_CONEXOES = 5

def excede_limite(place_id):
  return len(conexoes[place_id]) >= NUMERO_MAXIMO_DE_CONEXOES

def existe_conexao(place_id, proximo_id):
  return place_id in conexoes[proximo_id]

def pode_conectar(place_id, proximo_id):
  return (
    proximo_id != place_id and
    not excede_limite(place_id) and
    not excede_limite(proximo_id) and
    not existe_conexao(place_id, proximo_id) and
    not existe_conexao(proximo_id, place_id)
  )

In [25]:
# @title Tentando Reduzir a Hiperconectividade criando 5 conexoes (mais proximos) para cada ponto
conexoes = {place_id: [] for place_id in df_distancias.index}

# Conecta cada ponto aos seus três pontos mais próximos que não excedam o limite de conexões
for place_id in df_distancias.index:
    distancias_ordenadas = df_distancias.loc[place_id].sort_values()
    for proximo_id in distancias_ordenadas.index:
        if pode_conectar(place_id, proximo_id):
            conexoes[place_id].append(proximo_id)
            conexoes[proximo_id].append(place_id)

            if excede_limite(place_id):
                break

mapa = cria_mapa_com_pontos(todos_os_pontos=df_pontos, com_raio=False)
adiciona_conexoes_em_um_mapa_com_pontos(mapa, conexoes, 2)
mapa.save("grafo_sem_hiper-conectividade_1.html")
mapa

In [26]:
# @title Tentando Reduzir a Hiper-conectividade criando 5 conexoes (mais proximos) para cada ponto partindo da sede
from collections import deque

conexoes = {place_id: [] for place_id in df_distancias.index}

fila = deque(["ID_GERADO"])
visited = []

while fila:
    place_id = fila.popleft()
    distancias_ordenadas = df_distancias.loc[place_id].sort_values().index
    visited.append(place_id)
    for proximo_id in distancias_ordenadas:  # Considerar apenas os mais próximos
        if proximo_id not in visited:
            conexoes[place_id].append(proximo_id)
            conexoes[proximo_id].append(place_id)
            fila.append(proximo_id)  # Adiciona o próximo ponto à fila para ser explorado

            if excede_limite(place_id):
              break

mapa = cria_mapa_com_pontos(todos_os_pontos=df_pontos, com_raio=False)
adiciona_conexoes_em_um_mapa_com_pontos(mapa, conexoes, 2)
mapa.save("grafo_sem_hiper-conectividade_2.html")
mapa